In [26]:
import json
# with open('./DAIL-SQL/dataset/process/SPIDER-TEST_SQL_0-SHOT_CTX-200_ANS-2048/questions.json' , 'r') as f:
#     generated_prompts_file_byte = f.read()
#     generated_prompts = json.loads(generated_prompts_file_byte)

with open('./codeS_pred/ENSEMBLE_seqLevelVote_SQLscore_BIRD-TEST_SQL_1_3_5-SHOT_EUCDISQUESTIONMASK_QA-EXAMPLE_CTX-200_ANS-2048_codeS_1b.json' , 'r') as f:
    generated_response_file_byte = f.read()
    generated_response = json.loads(generated_response_file_byte)

In [7]:
with open('./llama_pred/BIRD-TEST_SQL_0-SHOT_CTX-200_ANS-2048_evidence_Llama_7b.txt' , 'w')as f:
    for i in generated_response['questions']:
        f.write( i['response']+'\n' )

In [17]:
#Code to make the dataset (DAIL style dataset) from CodeS predictions and the generated prompts
with open('./llama_pred/BIRD-TEST_SQL_5-SHOT_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_Llama_7b.json' , 'r') as f:
    file_bytes = f.read()
    reference_json = json.loads(file_bytes)

print(reference_json['questions'][0].keys())


with open('./codeS_pred/pred_codes-1b_BIRD_table_num_5_column_num_6_5-shot_max_tokens_8192_max_new_tokens_256.json' , 'r') as f:
    file_bytes = f.read()
    pred_json = json.loads(file_bytes)
i = 0

with open('./codeS_pred/prompts_codes-1b_BIRD_table_num_5_column_num_6_5-shot_max_tokens_8192_max_new_tokens_256.json' , 'r') as f:
    file_bytes = f.read()
    prompts_json = json.loads(file_bytes)

for i in pred_json.keys():
    response = pred_json[i]
    prompts = prompts_json[i]
    reference_json['questions'][int(i)]['prompt'] = prompts
    reference_json['questions'][int(i)]['response'] = response.split('----- bird -----')[0]

with open('./codeS_pred/codes-1b_BIRD_table_num_5_column_num_6_5-shot_max_tokens_8192_max_new_tokens_256.json' , 'w' )as f:
    json.dump(reference_json , f)

dict_keys(['prompt_tokens', 'prompt', 'response', 'n_examples', 'db_id'])


In [68]:
import pickle as pkl
with open('output_sequences-1.pkl', 'rb') as f:  # open a text file
    output_sequences = pkl.load(f)

In [1]:
import json
import time
def extract_prompt_from_list_of_questions(question_list):
    batch_list = []
    for i in question_list:
        batch_list.append(i['prompt'])
    return batch_list

from utils.post_process import get_exec_output

import psutil
def query_to_db(query , db_id , db_dir ):
    db_path = f"{db_dir}/{db_id}/{db_id}"
    flag, denotation = get_exec_output(
            db_path,
            query)
    
    return flag, denotation

from threading import Thread
import threading
import ctypes
class Thread_with_exception(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None , daemon=False):
        Thread.__init__(self, group, target, name, args, kwargs , daemon=daemon)
        
        self._return = None
            
    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
         
    def get_id(self):
        # returns id of the respective thread
        if hasattr(self, '_thread_id'):
            return self._thread_id
        for id, thread in threading._active.items():
            if thread is self:
                return id

    def join(self, *args):
        Thread.join(self, *args)
        return self._return
 
    def raise_exception(self):
        thread_id = self.get_id()
        res = ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, ctypes.py_object(SystemExit))
        if res > 1:
            ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, 0)
            print('Exception raise failure')
            

import time
from utils.post_process import result_eq

def is_queries_equal(testing_query , ground_truth_query , db_id , db_dir, timeout_time , gt_results = None):
    #Input:
        #time_out_time: integer: time in seconds
    
    if ground_truth_query!='':
        with open('./log.txt', 'a') as f:
            f.write(f"procceing the ground_truth_query:\n{ground_truth_query}\n")
        start_time = time.time()
        
        gt_flag, gt_denotation = query_to_db(ground_truth_query , db_id , db_dir)
    
        gt_process_time = time.time() - start_time
        with open('./log.txt', 'a') as f:
            f.write(f"processing time: {gt_process_time}\n\n")
        gt_results = (gt_flag, gt_denotation , gt_process_time)
        
    else:
        gt_flag, gt_denotation, gt_process_time = gt_results

    max_timeout_time = max( timeout_time , 2*gt_process_time )

    test_thread = Thread_with_exception(target= query_to_db , args = (testing_query , db_id , db_dir ) , daemon=True )
    with open('./log.txt', 'a') as f:
        f.write(f"proccessing the testing_query:\n{testing_query}\n")
    start_time = time.time()
    test_thread.start()
    test_result = test_thread.join(max_timeout_time)
    if test_thread.is_alive():
        with open('./log.txt', 'a') as f:
            f.write(f"**********processing is terminated due to timeout. max_timeout_time: {max_timeout_time}\n")
        test_thread.raise_exception()
        test_thread.join()
        return False, gt_results
        
    test_flag, test_denotation = test_result
    with open('./log.txt', 'a') as f:
        f.write(f"Processing time for the testing query: {time.time() - start_time}\n\n")
    
    if gt_flag[0] != 'result':
        with open('./log.txt', 'a') as f:
            f.write(f"!!!!!!!!!!The following ground truth has an error:\n{ground_truth_query}\n\n")
        return False, gt_results
    elif test_flag[0] != 'result':
        return False, gt_results
    elif 'ORDER BY' in ground_truth_query or 'order by' in ground_truth_query:
        is_equal = result_eq(gt_flag[1] , test_flag[1] , order_matters=True)
    else:
        is_equal = result_eq(gt_flag[1] , test_flag[1] , order_matters=False)
    
    return is_equal, gt_results
    
import os
def put_responses_back_to_json_dataset(index , json_dataset , sequences, dataset_type='spider' , timeout_time=60):#dataset_type=spider/bird
    #Inputs:
        #sequences: List of [ { 'generated_text' : gen_text } ] or [gen_text1 , gen_text2 , ...]
        #json_dataset: json dataset that contains the ground truth in its 'response' part
            #If the elements in input sequences are like seq[0]['generated_text'] then json_dataset should contain the prompt in 'prompts' part
    gt_result_cache_file = './cache/' + dataset_type + '_results.pkl'
    gt_results_is_cached = False
    gt_results_list = []
    
    if os.path.isfile(gt_result_cache_file):
        gt_results_is_cached = True
        with open(gt_result_cache_file , 'rb' )as f:
            gt_results_list = pkl.load(f)
    
    db_dir = './DAIL-SQL/dataset/'+ dataset_type +'/database'
    execution_accuracy = 0
    
    for i in range( 0, len(sequences), 1 ):
        seq = sequences[i]
        # print(f"Number of processed sequences: {i}\t|\tNumber of correct queries: {execution_accuracy} \n")
        # if i%10==0:
        with open('./log.txt', 'a') as f:
            f.write(f"Number of processed sequences: {i}\t|\tNumber of correct queries: {execution_accuracy} \n")
            
        prompt_len = len ( json_dataset['questions'][index+i]['prompt'] )
        if isinstance(seq, list):
            gen_text = seq[0]['generated_text'][prompt_len:]
        else:
            # gen_text = seq[prompt_len:]
            gen_text = seq
            
        processed_gen_text = post_process_get_sql_from_gentext(gen_text)
        
        db_id = json_dataset['questions'][index+i]['db_id']

        if gt_results_is_cached:
            is_equal , gt_results = is_queries_equal(processed_gen_text , '' , db_id , db_dir, timeout_time , gt_results = gt_results_list[i])
        else:
            ground_truth = post_process_get_sql_from_gentext( json_dataset['questions'][index+i]['response'] )
            is_equal , gt_results = is_queries_equal(processed_gen_text , ground_truth , db_id , db_dir, timeout_time )
            gt_results_list.append(gt_results)
        
        json_dataset['questions'][index+i]['response'] = processed_gen_text
        execution_accuracy += is_equal
        # print('is_equal: ', is_equal)
        # print('--------------------------')
    if not gt_results_is_cached:
        with open(gt_result_cache_file , 'wb' )as f:
            pkl.dump( gt_results_list , f )
        
    return execution_accuracy

from utils.post_process import process_duplication
def post_process_get_sql_from_gentext(gen_text):
    # remove \n and extra spaces
    sql = " ".join(gen_text.replace("\n", " ").split())
    sql = process_duplication(sql)
    # python version should >= 3.8
    if sql.startswith("SELECT"):
        sql = sql
    elif sql.startswith(" "):
        sql = "SELECT" + sql
    else:
        sql = "SELECT " + sql
    return sql

data_size=1034
def eval_list_sql(sql_list , groundtruth_json_file_name , output_filename='' , dataset_type='spider' ):
    #This function gets a list of sql predictions and evalueates it and creates a dail style dataset if the output_file_name is given
    # groundtruth_json_file_name: Address of a dail style dataset containing the groundtruth in its response part.
        #If the sql_list elements are of shape seq[0]['generated_text'] groundtruth_json_file_name should have the prompts too
    with open(groundtruth_json_file_name , 'r') as f:
        generated_prompts_file_byte = f.read()
        generated_prompts = json.loads(generated_prompts_file_byte)
        exec_acc = put_responses_back_to_json_dataset( 0 , generated_prompts , sql_list , dataset_type=dataset_type )
    print('execution accuracy = ' , exec_acc/data_size)
    with open('./log.txt', 'a') as f:
        f.write(f"execution accuracy = {exec_acc/data_size} \n")
    if output_filename !='':
        with open( output_filename , 'w' )as f:
            json.dump(generated_prompts , f)

def dail_dataset_to_response_list( dail_style_dataset_name ):
    #This function gets a dail-style dataset and returns the all the responses in that dataset in a list.
    return_list = []
    with open( dail_style_dataset_name , 'r') as f:
        generated_response_file_byte = f.read()
        generated_response = json.loads(generated_response_file_byte)
    for i in generated_response['questions']:
        return_list.append( i['response'] )
    return return_list
        
    

In [192]:
#re-evaluating a dataset that is created before
import json
groundtruth_json_file_name = './DAIL-SQL/dataset/process/SPIDER-TEST_SQL_0-SHOT_CTX-200_ANS-2048/questions.json'

evaluating_dataset = './llama_pred/SPIDER_beam_4_generate-TEST_SQL_3-SHOT_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_llama_7b.json'

output_sequences = dail_dataset_to_response_list( evaluating_dataset )

eval_list_sql(output_sequences , groundtruth_json_file_name , output_filename='' , dataset_type='spider' )

execution accuracy =  0.30077369439071566


In [76]:
#evaluating some generated prompts recently generated.
import pickle as pkl

groundtruth_json_file_name = './DAIL-SQL/dataset/process/SPIDER-TEST_SQL_3-SHOT_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_llama_7b/questions.json'

# with open('./output_sequences-5_lenpen1.pkl', 'rb') as f:  # open a text file
#     output_sequences = pkl.load(f)

eval_list_sql(output_sequences , groundtruth_json_file_name , 
              output_filename='./llama_pred/SPIDER_beam_4_lenpen0-TEST_SQL_3-SHOT_9-12_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_llama_7b.json' , dataset_type='spider' )


execution accuracy =  0.3578336557059961


In [75]:
#grouping the pieces of output_sequences
import pickle as pkl
prefix = './outputs2/output_sequences_'
output_sequences = []
for i in range(0 , 1050 , 50):
    if i <1000:
        if i == 0: 
            file_name = prefix + '0_'  + str(i+50).lstrip('0') + '-4.pkl'
        else:
            file_name = prefix + str(i).lstrip('0') + '_' + str(i+50).lstrip('0') + '-4.pkl'
    else:
        file_name = prefix + str(i).lstrip('0') + '_end' + '-4.pkl'
    print(file_name)
    with open(file_name , 'rb') as f:
        part_of_output = pkl.load(f)
    output_sequences.extend(part_of_output)

print( len(output_sequences) )

./outputs2/output_sequences_0_50-4.pkl
./outputs2/output_sequences_50_100-4.pkl
./outputs2/output_sequences_100_150-4.pkl
./outputs2/output_sequences_150_200-4.pkl
./outputs2/output_sequences_200_250-4.pkl
./outputs2/output_sequences_250_300-4.pkl
./outputs2/output_sequences_300_350-4.pkl
./outputs2/output_sequences_350_400-4.pkl
./outputs2/output_sequences_400_450-4.pkl
./outputs2/output_sequences_450_500-4.pkl
./outputs2/output_sequences_500_550-4.pkl
./outputs2/output_sequences_550_600-4.pkl
./outputs2/output_sequences_600_650-4.pkl
./outputs2/output_sequences_650_700-4.pkl
./outputs2/output_sequences_700_750-4.pkl
./outputs2/output_sequences_750_800-4.pkl
./outputs2/output_sequences_800_850-4.pkl
./outputs2/output_sequences_850_900-4.pkl
./outputs2/output_sequences_900_950-4.pkl
./outputs2/output_sequences_950_1000-4.pkl
./outputs2/output_sequences_1000_end-4.pkl
1034


In [63]:
print(output_sequences[751])

count(*) FROM Country AS T1 JOIN CountryLanguage AS T2 ON T1.id  =  T2.country_id WHERE T2.language  =  "Chinese"




In [56]:
#getting the output_seqs from logs:
file_name = './logs1/log_50_100-3.txt'
output_list = [ [] for i in range(10) ]
with open(file_name , 'r') as f:
    lines = f.readlines()
seen_thread = False
for line in lines:
    if line.startswith('Thread_num:'):
        thread_index = int(line[11])
        seen_thread = True
    if seen_thread==True and line.startswith('The chosen gen text:  ['):
        output_list[thread_index].append(line[24:-2])
        print(line[24:-3])
        print( line[24:-3].encode().decode('unicode_escape') )
        seen_thread==False
for index , thread in enumerate(output_list):
    print(f'{index}- {len(thread)}')

count(DISTINCT PetType) FROM Has_Pet\n\n', 'count(DISTINCT PetType) FROM Has_Pet\n\n
count(DISTINCT PetType) FROM Has_Pet

', 'count(DISTINCT PetType) FROM Has_Pet


PetType, MAX(weight) FROM Pets GROUP BY PetType\n\n', 'count(*) FROM Has_Pet WHERE StuID > 20\n\n
PetType, MAX(weight) FROM Pets GROUP BY PetType

', 'count(*) FROM Has_Pet WHERE StuID > 20


StuID, weight FROM Has_Pet WHERE PetID >  1\n\n', 'avg(pet_age) ,  max(pet_age) FROM Has_Pet\n\n
StuID, weight FROM Has_Pet WHERE PetID >  1

', 'avg(pet_age) ,  max(pet_age) FROM Has_Pet


avg(age) FROM Student WHERE StuID NOT IN (SELECT StuID FROM Has_Pet)\n\n', 'avg(age) FROM Student WHERE StuID NOT IN (SELECT StuID FROM Has_Pet)\n\n
avg(age) FROM Student WHERE StuID NOT IN (SELECT StuID FROM Has_Pet)

', 'avg(age) FROM Student WHERE StuID NOT IN (SELECT StuID FROM Has_Pet)


count(*) FROM continents\n\n', 'count(*) FROM continents\n\n
count(*) FROM continents

', 'count(*) FROM continents


ContId, Continent, COUNT(*) FROM contine

In [154]:
output_sequences = []
output_sequences.extend(output_list[0])
output_sequences.extend(output_list[1])
output_sequences.extend(output_list[2])
output_sequences.extend(output_list[3])
output_sequences.extend(output_list[4])
output_sequences.extend(output_list[5])
output_sequences.extend(output_list[6])
output_sequences.extend(output_list[7])
output_sequences.extend(output_list[8])
output_sequences.extend(output_list[9])

In [155]:
with open('./output_seq/output_sequences_800_end-4.pkl' , 'wb') as f:
    pkl.dump(output_sequences , f)

In [153]:
import pickle as pkl
with open('output_sequences_test1.pkl', 'rb') as f:  # open a text file
    part_of_seq = pkl.load(f)
output_list[9].extend(part_of_seq)
for index , thread in enumerate(output_list):
    print(f'{index}- {len(thread)}')

0- 23
1- 23
2- 23
3- 23
4- 23
5- 23
6- 23
7- 23
8- 23
9- 27


In [144]:
output_list[6] = output_list[6][:16]